In [205]:
import random
class Entity:
    def __init__(self, genome_len, genome=None):
        self.genome_len = genome_len
        if genome is None:
            self.genome = [random.randint(0, 1) for i in range(genome_len)]
        else:
            self.genome = genome
        self.phenotype = self.set_phenotype()
        self.fitness = self.set_fitness()
        self.probability = 0
        
    def __str__(self):
        return f'Phenotype: {self.phenotype}, Genotype: {self.genome}, Fitness: {self.fitness}'
    
    def __lt__(self, other):
        if type(other) is Entity:
            return self.probability < other.probability
        return NotImplemented
    
    def set_phenotype(self):
        val = 0
        exp = self.genome_len - 1
        for i in range(self.genome_len):
            val += (self.genome[i] * 2**(exp - i))
        return val
    
    def set_fitness(self):
        return  self.phenotype**2
    
    def set_probability(self, probability):
        self.probability = probability
        
    @staticmethod
    def crossover(parent1, parent2):
        rnd = random.randint(0, parent1.genome_len - 1)
        new_genome = list()
        for i in range(rnd + 1):
            new_genome.append(parent1.genome[i])
        for i in range(rnd + 1, parent1.genome_len):
            new_genome.append(parent2.genome[i])
        return new_genome
    
    @staticmethod
    def mutation(genome, mutation_rate):
        for i in range(len(genome)):
            if mutation_rate > random.random():
                if genome[i] == 0:
                    genome[i] = 1
                else:
                    genome[i] = 0
        return genome
        

In [240]:
import random
class Population:
    def __init__(self, population_len, genome_len, mutation_rate):
        self.population_len = population_len
        self.genome_len = genome_len
        self.mutation_rate = mutation_rate
        self.generation = -1
        self.entities = [Entity(genome_len) for i in range(population_len)]
        self.define_population(self.entities)
    
    def define_population(self, entities):
        self.generation += 1
        self.entities = entities
        self.assign_probability()    
        self.entities.sort(reverse=True)

    def __str__(self):
        strs = list()
        strs.append(f'Generation: {self.generation}')
        # strs.extend([str(self.entities[i]) for i in range(self.population_len)])
        # strs.append(f'Total fitness: {self.get_total_fitness()}')
        strs.append(f'Avg fitness: {self.get_avg_fitness()}')
        strs.append(f'Max fitness: {self.get_max_fitness()}')
        return '\n'.join(strs)
    
    def __iter__(self):
        self.current = -1
        return self
    
    def __next__(self):
        self.current += 1
        if self.current < self.population_len:
            return self.entities[self.current]
        raise StopIteration
    
    def __getitem__(self, item):
        return self.entities[item]
    
    def get_total_fitness(self):
        return sum(entity.fitness for entity in self.entities)
    
    def get_avg_fitness(self):
        return self.get_total_fitness()/self.population_len
        
    def get_max_fitness(self):
        return max(entity.fitness for entity in self.entities)
    
    def assign_probability(self):
        epsilon = 10**-9
        total_fitness = self.get_total_fitness()
        for entity in self.entities:
            # entity.set_probability((entity.get_fitness() + epsilon) / total_fitness)
            entity.set_probability(entity.fitness / total_fitness)
                
    def evolve(self):
        probabilities_list = [e.probability for e in self.entities]
        for i in range(1, self.population_len):
            probabilities_list[i] += probabilities_list[i - 1]
        new_entities = list()
        for i in range(self.population_len):
            p1, p2 = self.select_parents(probabilities_list)
            new_genome = Entity.crossover(self.entities[p1], self.entities[p2])
            new_genome = Entity.mutation(new_genome, self.mutation_rate)
            new_entities.append(Entity(self.genome_len, new_genome))
        self.define_population(new_entities)
         
    def evolve_for_generations(self, generations):
        for i in range(generations):
            self.evolve()
            #print(self)
            
            
    #If parent is already selected, 0.5p to select the left or right   
    @staticmethod
    def select_parents(probabilities_list):
        rnd = random.random()
        p1 = 0
        while probabilities_list[p1] < rnd:
            p1+=1
        rnd = random.random()
        p2 = 0
        while probabilities_list[p2] < rnd:
            p2+=1
        if p2 == p1:
            if p2 == len(probabilities_list) - 1:
                p2 -= 1
            elif p2 == 0:
              p2 += 1
            else:
                rnd = random.random()
                if rnd > 0.5:
                    p2 += 1
                else:
                    p2 -= 1
        return p1, p2
        

In [251]:
population_l = 20
genome_l = 5
mutation_rate = .01
population = Population(population_l, genome_l, mutation_rate)
print(population)
population.evolve_for_generations(500)
print(population)

Generation: 0
Avg fitness: 310.3
Max fitness: 961
Generation: 500
Avg fitness: 858.8
Max fitness: 961


In [254]:
population_l = 20
mutation_rate = .01
generations = 20
tests = 1000
avg_sum = 0
max_sum = 0
for i in range(tests):
    population = Population(population_l, genome_l, mutation_rate)
    population.evolve_for_generations(generations)
    print("AVG:", population.get_avg_fitness(), "MAX:", population.get_max_fitness())
    avg_sum += population.get_avg_fitness()
    max_sum += population.get_max_fitness()
print("Average values: AVG:", avg_sum/tests, "MAX:", max_sum/tests)

AVG: 857.8 MAX: 961
AVG: 955.0 MAX: 961
AVG: 886.55 MAX: 961
AVG: 821.05 MAX: 841
AVG: 961.0 MAX: 961
AVG: 875.45 MAX: 961
AVG: 936.6 MAX: 961
AVG: 905.45 MAX: 961
AVG: 869.85 MAX: 900
AVG: 936.6 MAX: 961
AVG: 847.25 MAX: 961
AVG: 933.55 MAX: 961
AVG: 961.0 MAX: 961
AVG: 903.15 MAX: 961
AVG: 957.95 MAX: 961
AVG: 878.35 MAX: 961
AVG: 876.75 MAX: 961
AVG: 831.25 MAX: 961
AVG: 803.55 MAX: 961
AVG: 924.15 MAX: 961
AVG: 763.8 MAX: 961
AVG: 866.75 MAX: 961
AVG: 898.75 MAX: 961
AVG: 755.45 MAX: 841
AVG: 867.6 MAX: 900
AVG: 915.55 MAX: 961
AVG: 933.95 MAX: 961
AVG: 849.35 MAX: 961
AVG: 924.95 MAX: 961
AVG: 900.6 MAX: 961
AVG: 869.6 MAX: 961
AVG: 882.6 MAX: 900
AVG: 906.4 MAX: 961
AVG: 910.75 MAX: 961
AVG: 758.0 MAX: 900
AVG: 954.9 MAX: 961
AVG: 786.35 MAX: 961
AVG: 909.2 MAX: 961
AVG: 847.05 MAX: 961
AVG: 794.0 MAX: 961
AVG: 826.8 MAX: 961
AVG: 864.6 MAX: 961
AVG: 924.25 MAX: 961
AVG: 934.85 MAX: 961
AVG: 936.6 MAX: 961
AVG: 939.95 MAX: 961
AVG: 846.55 MAX: 961
AVG: 880.35 MAX: 961
AVG: 939.4 

In [256]:
population_l = 20
mutation_rate = .03
generations = 20
tests = 1000
avg_sum = 0
max_sum = 0
for i in range(tests):
    population = Population(population_l, genome_l, mutation_rate)
    population.evolve_for_generations(generations)
    print("AVG:", population.get_avg_fitness(), "MAX:", population.get_max_fitness())
    avg_sum += population.get_avg_fitness()
    max_sum += population.get_max_fitness()
print("Average values: AVG:", avg_sum/tests, "MAX:", max_sum/tests)

AVG: 856.8 MAX: 961
AVG: 864.2 MAX: 961
AVG: 840.85 MAX: 961
AVG: 910.5 MAX: 961
AVG: 830.15 MAX: 961
AVG: 761.35 MAX: 841
AVG: 902.75 MAX: 961
AVG: 852.3 MAX: 961
AVG: 857.85 MAX: 961
AVG: 827.75 MAX: 961
AVG: 733.6 MAX: 961
AVG: 886.6 MAX: 961
AVG: 818.0 MAX: 961
AVG: 866.5 MAX: 961
AVG: 783.5 MAX: 961
AVG: 784.35 MAX: 961
AVG: 780.05 MAX: 961
AVG: 860.25 MAX: 961
AVG: 653.55 MAX: 961
AVG: 856.35 MAX: 961
AVG: 948.9 MAX: 961
AVG: 738.85 MAX: 900
AVG: 729.05 MAX: 961
AVG: 787.6 MAX: 961
AVG: 856.75 MAX: 961
AVG: 837.35 MAX: 961
AVG: 884.8 MAX: 961
AVG: 737.1 MAX: 900
AVG: 805.7 MAX: 961
AVG: 919.0 MAX: 961
AVG: 875.8 MAX: 961
AVG: 823.15 MAX: 961
AVG: 803.15 MAX: 961
AVG: 855.3 MAX: 961
AVG: 786.4 MAX: 961
AVG: 925.8 MAX: 961
AVG: 814.85 MAX: 961
AVG: 842.5 MAX: 961
AVG: 925.6 MAX: 961
AVG: 905.95 MAX: 961
AVG: 758.2 MAX: 961
AVG: 646.6 MAX: 961
AVG: 862.55 MAX: 961
AVG: 841.45 MAX: 961
AVG: 873.9 MAX: 961
AVG: 847.3 MAX: 961
AVG: 842.25 MAX: 961
AVG: 856.5 MAX: 961
AVG: 782.1 MAX: 96

In [258]:
population_l = 20
mutation_rate = .03
generations = 100
tests = 1000
avg_sum = 0
max_sum = 0
for i in range(tests):
    population = Population(population_l, genome_l, mutation_rate)
    population.evolve_for_generations(generations)
    print("AVG:", population.get_avg_fitness(), "MAX:", population.get_max_fitness())
    avg_sum += population.get_avg_fitness()
    max_sum += population.get_max_fitness()
print("Average values: AVG:", avg_sum/tests, "MAX:", max_sum/tests)

AVG: 872.7 MAX: 961
AVG: 777.55 MAX: 961
AVG: 850.1 MAX: 961
AVG: 912.45 MAX: 961
AVG: 876.6 MAX: 961
AVG: 878.9 MAX: 961
AVG: 881.35 MAX: 961
AVG: 832.9 MAX: 961
AVG: 896.8 MAX: 961
AVG: 799.05 MAX: 961
AVG: 903.15 MAX: 961
AVG: 932.1 MAX: 961
AVG: 781.75 MAX: 961
AVG: 740.65 MAX: 961
AVG: 781.0 MAX: 900
AVG: 886.45 MAX: 961
AVG: 837.0 MAX: 961
AVG: 834.8 MAX: 961
AVG: 897.05 MAX: 961
AVG: 835.45 MAX: 961
AVG: 868.2 MAX: 961
AVG: 824.35 MAX: 961
AVG: 843.8 MAX: 961
AVG: 789.05 MAX: 961
AVG: 765.75 MAX: 961
AVG: 915.8 MAX: 961
AVG: 709.95 MAX: 961
AVG: 817.55 MAX: 961
AVG: 756.15 MAX: 961
AVG: 864.15 MAX: 961
AVG: 866.8 MAX: 961
AVG: 928.75 MAX: 961
AVG: 879.3 MAX: 961
AVG: 868.45 MAX: 961
AVG: 815.15 MAX: 961
AVG: 743.55 MAX: 961
AVG: 766.55 MAX: 961
AVG: 909.05 MAX: 961
AVG: 786.3 MAX: 961
AVG: 836.7 MAX: 961
AVG: 856.9 MAX: 961
AVG: 847.7 MAX: 961
AVG: 736.4 MAX: 961
AVG: 809.25 MAX: 900
AVG: 856.55 MAX: 961
AVG: 814.6 MAX: 961
AVG: 865.8 MAX: 961
AVG: 873.6 MAX: 961
AVG: 806.0 MAX:

In [260]:
population_l = 50
mutation_rate = .03
generations = 20
tests = 1000
avg_sum = 0
max_sum = 0
for i in range(tests):
    population = Population(population_l, genome_l, mutation_rate)
    population.evolve_for_generations(generations)
    print("AVG:", population.get_avg_fitness(), "MAX:", population.get_max_fitness())
    avg_sum += population.get_avg_fitness()
    max_sum += population.get_max_fitness()
print("Average values: AVG:", avg_sum/tests, "MAX:", max_sum/tests)

AVG: 797.96 MAX: 961
AVG: 857.72 MAX: 961
AVG: 878.96 MAX: 961
AVG: 871.26 MAX: 961
AVG: 812.8 MAX: 961
AVG: 891.12 MAX: 961
AVG: 841.22 MAX: 961
AVG: 816.02 MAX: 961
AVG: 840.26 MAX: 961
AVG: 870.42 MAX: 961
AVG: 842.12 MAX: 961
AVG: 776.14 MAX: 961
AVG: 819.72 MAX: 961
AVG: 824.08 MAX: 961
AVG: 850.84 MAX: 961
AVG: 880.78 MAX: 961
AVG: 834.62 MAX: 961
AVG: 858.14 MAX: 961
AVG: 802.76 MAX: 961
AVG: 866.92 MAX: 961
AVG: 815.02 MAX: 961
AVG: 847.22 MAX: 961
AVG: 812.9 MAX: 961
AVG: 836.1 MAX: 961
AVG: 818.7 MAX: 961
AVG: 862.86 MAX: 961
AVG: 831.0 MAX: 961
AVG: 797.16 MAX: 961
AVG: 854.56 MAX: 961
AVG: 837.16 MAX: 961
AVG: 801.78 MAX: 961
AVG: 843.44 MAX: 961
AVG: 789.62 MAX: 961
AVG: 831.28 MAX: 961
AVG: 812.82 MAX: 961
AVG: 859.22 MAX: 961
AVG: 887.0 MAX: 961
AVG: 841.48 MAX: 961
AVG: 841.64 MAX: 961
AVG: 791.82 MAX: 961
AVG: 846.66 MAX: 961
AVG: 864.38 MAX: 961
AVG: 769.82 MAX: 961
AVG: 748.52 MAX: 961
AVG: 848.74 MAX: 961
AVG: 829.32 MAX: 961
AVG: 885.24 MAX: 961
AVG: 897.4 MAX: 961

In [261]:
population_l = 50
mutation_rate = .01
generations = 20
tests = 1000
avg_sum = 0
max_sum = 0
for i in range(tests):
    population = Population(population_l, genome_l, mutation_rate)
    population.evolve_for_generations(generations)
    print("AVG:", population.get_avg_fitness(), "MAX:", population.get_max_fitness())
    avg_sum += population.get_avg_fitness()
    max_sum += population.get_max_fitness()
print("Average values: AVG:", avg_sum/tests, "MAX:", max_sum/tests)

AVG: 956.16 MAX: 961
AVG: 900.86 MAX: 961
AVG: 905.4 MAX: 961
AVG: 908.6 MAX: 961
AVG: 896.68 MAX: 961
AVG: 894.72 MAX: 961
AVG: 848.06 MAX: 961
AVG: 886.5 MAX: 961
AVG: 892.04 MAX: 961
AVG: 893.44 MAX: 961
AVG: 921.0 MAX: 961
AVG: 897.72 MAX: 961
AVG: 907.58 MAX: 961
AVG: 866.5 MAX: 961
AVG: 867.92 MAX: 961
AVG: 858.48 MAX: 961
AVG: 889.64 MAX: 961
AVG: 900.06 MAX: 961
AVG: 911.62 MAX: 961
AVG: 906.56 MAX: 961
AVG: 938.24 MAX: 961
AVG: 913.02 MAX: 961
AVG: 871.3 MAX: 961
AVG: 927.7 MAX: 961
AVG: 936.86 MAX: 961
AVG: 895.84 MAX: 961
AVG: 933.72 MAX: 961
AVG: 947.66 MAX: 961
AVG: 918.02 MAX: 961
AVG: 909.44 MAX: 961
AVG: 888.74 MAX: 961
AVG: 913.72 MAX: 961
AVG: 937.86 MAX: 961
AVG: 927.14 MAX: 961
AVG: 912.9 MAX: 961
AVG: 927.08 MAX: 961
AVG: 896.46 MAX: 961
AVG: 906.14 MAX: 961
AVG: 884.44 MAX: 961
AVG: 903.7 MAX: 961
AVG: 905.7 MAX: 961
AVG: 923.34 MAX: 961
AVG: 925.72 MAX: 961
AVG: 889.02 MAX: 961
AVG: 859.1 MAX: 961
AVG: 817.36 MAX: 961
AVG: 923.74 MAX: 961
AVG: 919.92 MAX: 961
AVG